# Embedded bokeh server: about sessions life cycle management

In [ ]:
from __future__ import print_function
from collections import deque
from math import ceil, pi
import socket
import six
import ipywidgets as widgets
import numpy as np
from IPython.display import HTML, clear_output
from tornado.ioloop import IOLoop
from bokeh.application import Application
from bokeh.application.handlers import Handler, FunctionHandler
from bokeh.embed import autoload_server
from bokeh.io import output_notebook, reset_output
from bokeh.layouts import row, column, layout, gridplot, widgetbox
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.glyphs import Rect
from bokeh.plotting import figure
from bokeh.plotting.figure import Figure
from bokeh.server.server import Server

In [ ]:
output_notebook()

### BokehSessionHandler class
See 'Lifecycle' in [bokeh server architecture](http://bokeh.pydata.org/en/latest/docs/dev_guide/server.html#devguide-server).
Not really used so far but might be useful in the future.

In [ ]:
class BokehSessionHandler(Handler):

    def on_server_loaded(self, server_context):
        print("SessionHandler: on_server_loaded <<")
        print("SessionHandler: on_server_loaded >>")

    def on_server_unloaded(self, server_context):
        print("SessionHandler: on_server_unloaded <<")
        print("SessionHandler: on_server_unloaded >>")

    def on_session_created(self, session_context):
        print("SessionHandler: on_session_created <<")
        BokehServer.print_info(True)
        print("SessionHandler: on_session_created >>")

    def on_session_destroyed(self, session_context):
        print("SessionHandler: on_server_unloaded <<")
        print("SessionHandler: on_server_unloaded >>")


### BokehSession class
Basically, each bokeh session 'serves' a `BokehSession`. This is a super class that could be extend in the future.

See `BokehServer.serve`.

In [ ]:
class BokehSession(object):
    
    def __init__(self):
        """the associated bokeh document"""
        self._doc = None
        """periodic callback period in seconds - defaults to None (i.e. callback disabled)"""
        self._period = None
        
    def setup_model(self):
        """return the session model or None is no model"""
        return None

    def periodic_callback(self):
        """return the periodic callback or None is no periodic activity"""
        return None
    
    def callback_period(self):
        """return the (periodic) callback period in seconds"""
        return self._period
    

### BokehServer class
Embbded bokeh server singleton. 
Usage: simply call `BokehServer.open_session` passing a properly filled `BokehSessionForm`.

In [ ]:
class BokehServer(object):

    __bkh_app__ = None
    __bkh_srv__ = None
    __srv_url__ = None
    __sessions__ = deque()
        
    @staticmethod
    def __start_server():
        app = Application(FunctionHandler(BokehServer.__entry_point))
        app.add(BokehSessionHandler())
        srv = Server(
            {'/': app},
            io_loop=IOLoop.instance(),
            port=0,
            host='*',
            allow_websocket_origin=['*']
        )
        srv.start()
        srv_addr = srv.address if srv.address else socket.gethostbyname(socket.gethostname())
        BokehServer.__bkh_srv__ = srv
        BokehServer.__bkh_app__ = app
        BokehServer.__srv_url__ = 'http://{}:{}'.format(srv_addr, srv.port)
        
    @staticmethod
    def __entry_point(doc):
        try:
            #TODO: should we locked BokehServer.__callbacks__?
            session = BokehServer.__sessions__.pop()
            session._doc = doc
            model = session.setup_model()
            if model:
                doc.add_root(model)
            BokehServer.__add_periodic_callback(session)
        except Exception as e:
            print(e)
        
    @staticmethod
    def __add_periodic_callback(session):
        assert(isinstance(session, BokehSession))
        pcb = session.periodic_callback
        try:
            session._doc.remove_periodic_callback(pcb)
        except:
            pass
        prd = session.callback_period()
        if prd is not None:
            session._doc.add_periodic_callback(pcb, max(250, 1000. * prd))
        
    @staticmethod
    def open_session(new_session):
        assert(isinstance(new_session, BokehSession))
        if not BokehServer.__bkh_srv__:
            BokehServer.__start_server()
        BokehServer.__sessions__.appendleft(new_session)
        script = autoload_server(model=None, url=BokehServer.__srv_url__)
        html_display = HTML(script)
        display(html_display)
        
    @staticmethod
    def close_session(session):
        assert(isinstance(session, BokehSession))
        session_id = session._doc.session_context.id
        print("trying to destroy session '{}'".format(session_id))
        session = BokehServer.__bkh_srv__.get_session('/', session_id)
        session.destroy()
        print("session '{}' successfully destroyed".format(session_id))
        
    @staticmethod
    def update_callback_period(session):
        assert(isinstance(session, BokehSession))
        BokehServer.__add_periodic_callback(session)
        
    @staticmethod
    def print_info(called_from_session_handler=False):
        if not BokehServer.__bkh_srv__:
            print("no Bokeh server running") 
            return
        try:
            print("Bokeh server URL: {}".format(BokehServer.__srv_url__))
            sessions = BokehServer.__bkh_srv__.get_sessions()
            num_sessions = len(sessions)
            if called_from_session_handler:
                num_sessions += 1
            print("Number of opened sessions: {}".format(num_sessions))
        except Exception as e:
            print(e)

### MySession class
A user specialization of `BokehSession`. Actual/concrete/... session implementation.

In [ ]:
from bokeh.models.widgets import Slider

class MySession(BokehSession):
    
    def __init__(self):
        BokehSession.__init__(self)
        self._period = 1.
        self._np = 100
        self._widgets_layout = None
        columns = dict()
        columns['x'] = self._gen_x_scale()
        columns['y'] = self._gen_random_data()
        self._cds = ColumnDataSource(data=columns)

    def _gen_x_scale(self):
        return np.linspace(1, self._np, num=self._np, endpoint=True)
    
    def _gen_random_data(self):
        return np.random.rand(self._np)
    
    def __on_refresh_period_change(self, attr, old, new):
        self._period = new
        BokehServer.update_callback_period(self)
        
    def __on_num_points_change(self, attr, old, new):
        self._np = int(new)

    def setup_model(self):
        rrs = Slider(start=0.25, end=2, value=self._period, step=0.25, title="Refresh period [s]")
        rrs.on_change("value", self.__on_refresh_period_change)
        nps = Slider(start=0, end=1000, value=self._np, step=10, title="Num. points")
        nps.on_change("value", self.__on_num_points_change)
        p = figure(plot_width=650, plot_height=200)
        p.toolbar_location = 'above'
        p.line(x='x', y='y', source=self._cds, color="navy", alpha=0.5)
        self._widgets_layout = widgetbox(nps, rrs)
        return layout([[self._widgets_layout, p]])
    
    def periodic_callback(self):
        self._cds.data.update(x=self._gen_x_scale(), y=self._gen_random_data())

### SC1: let's open a first session...

In [ ]:
s1 = MySession()
BokehServer.open_session(s1)

### SC2: open a second session...

In [ ]:
s2 = MySession()
BokehServer.open_session(s2)

### What if we re-execute SC1 and/or SC2?
We now have two running sessions and everything works properly. However, what if we re-execute SC1 or SC2?

Please, do so and see how many running sessions we have...

In [ ]:
BokehServer.print_info()

We now have 3 sessions running! It means that **re-executing the cell doesn't magically cleanup the previous session**. The same apply we if `clear` the cell output (see `Cell` menu > `Current Outputs` > `Clear`). 

The bad new is that things will clearly divergence after a few tens CS1 (and/or CS2) re-execution cause the zombies sessions continue to run in the background - generating some CPU load and memory leaks. 

So, the big question is: **is there a way to deal with this? a mechanism providing a way to properly cleanup a session when the cell to which it's attached is re-executed or cleared?**  

### Discussion

#### The Jupyter Notebook part of the problem
So far, the Notebook doesn't provide any notification mechanism that could help in our case [1]. There's no way to attach an action callback and some user data to a cell. We could imagine something like:
1. attach `my_cleanup_callback` to the _current cell_ for the `execute` and the `clear` actions
2. pass `my_internal_scheming_data` as an argument when `my_cleanup_callback` is triggered

With such a mechanism we could easily retrieve the cell content and properly release the associated resources.
There's certainly a smarter/more adapted/... solution but that's the idea. 

In our case, one could argue that adding a `close` button to model generated by `MySession.setup_model` could partially solve the problem. That's true as far as the user use this button to close the cell before re-executing it. IHMO, that's ugly and error prone. The `MyExtendedSession` class adds a close button in order to be able to work on the bokeh part of the problem (see below).  

#### The bokeh part of the problem
Let's pretend we have a way to be notified when it's time to cleanup our session(s). Ok, but, how to we properly cleanup a session? I'm afraid that, so far [2], there's none! Or I totally missed something.

In a [previous implementation](https://github.com/nleclercq/jupyter-for-controls/tree/master/bokeh-data-streaming-for-notebook), I'm using a _"one server per session per model"_ approach - i.e. a bokeh server is instanciated to serve a particular cell. With this design + a `close` button, it was easy to cleanup everything by destroying the server instance so that it's properly garbage-collected. Unfortunately, things seem to properly cleanup on python side but not on BokehJS (i.e. browser side). 
 
- [1] Jupyter Notebook <= 5.0.0
- [2] Bokeh <= 0.12.6 

### MyExtendedSession class

In [ ]:
from bokeh.models.widgets import Button

class MyExtendedSession(MySession):
    
    def __init__(self):
        MySession.__init__(self)

    def setup_model(self):
        model = super(MyExtendedSession, self).setup_model()
        b = Button(label='close')
        b.on_click(self.__on_close_clicked)
        self._widgets_layout.children.append(b)
        return model
    
    def periodic_callback(self):
        print('MyExtendedSession.periodic_callback <<')
        super(MyExtendedSession, self).periodic_callback()
        print('MyExtendedSession.periodic_callback >>')
        
    def __on_close_clicked(self):
        self.cleanup()
   
    def cleanup(self):  
        """try to cleanup everything properly"""
        try:
            # clear document content (i.e. remove roots)
            self._doc.clear()
        except Exception as e:
            self.exception(e)
        try:
            reset_output()
        except Exception as e:
            self.exception(e)
        try:  
            BokehServer.close_session(self)
        except Exception as e:
            self.exception(e) 
        clear_output()

In [ ]:
s3 = MyExtendedSession()
BokehServer.open_session(s3)

In [ ]:
BokehServer.print_info()